In [68]:
task = "drugrec"

In [69]:
import json

if task == "mortality" or task == "readmission":
    with open("/data/pj20/exp_data/ccscm_ccsproc_atc3/clusters_th015.json", "r") as f:
        clusters = json.load(f)
elif task == "lenofstay" or task == "drugrec":
    with open("/data/pj20/exp_data/ccscm_ccsproc/clusters_th015.json", "r") as f:
        clusters = json.load(f)

In [70]:
import numpy as np

def cosine_similarity(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [71]:
from get_emb import embedding_retriever

terms = None
if task == "mortality":
    terms = ['death', 'mortality', 'cause death', 'lead to death', 'high risk', "deadly"]
elif task == "readmission":
    terms = ['rehospitalization', 'readmission']
elif task == "lenofstay":
    terms = ["length of stay'", "bed days", "time in hospital"]
elif task == "drugrec":
    terms = ["drug recommendation", "prescription", "drug", "medication", "treatment"]

term_embs = []

for term in terms:
    term_embs.append(embedding_retriever(term))

tmp = {}

for clus in clusters.keys():
    tmp[clus] = 0
    for term_emb in term_embs:
        tmp[clus] += cosine_similarity(clusters[clus]['embedding'], term_emb)   


In [72]:
float(tmp['0'])

4.134378024520449

In [73]:
min_similarity = min(tmp.values())
max_similarity = max(tmp.values())

for clus in clusters.keys():
    tmp[clus] = (float(tmp[clus]) - min_similarity) / (max_similarity - min_similarity)


In [74]:
min_similarity

array([3.43326015])

In [75]:
for clus in clusters.keys():
    tmp[clus] = float(tmp[clus] ** 3)


In [76]:
max(tmp.values())

1.0

In [77]:
for clus in clusters.keys():
    if task == "mortality":
        clusters[clus]['attention_mortality'] = tmp[clus]
    elif task == "readmission":
        clusters[clus]['attention_readmission'] = tmp[clus]
    elif task == "lenofstay":
        clusters[clus]['attention_lenofstay'] = tmp[clus]
    elif task == "drugrec":
        clusters[clus]['attention_drugrec'] = tmp[clus]

In [78]:

if task == "mortality" or task == "readmission":

    with open("/data/pj20/exp_data/ccscm_ccsproc_atc3/clusters_th015.json", "w") as f:
        json.dump(clusters, f)

elif task == "lenofstay" or task == "drugrec":
    
        with open("/data/pj20/exp_data/ccscm_ccsproc/clusters_th015.json", "w") as f:
            json.dump(clusters, f)

In [ ]:
import pickle
import numpy as np

if task == "mortality" or task == "readmission":
    attn_file = f"/data/pj20/exp_data/ccscm_ccsproc_atc3/attention_weights_{task}.pkl"
elif task == "lenofstay" or task == "drugrec":
    attn_file = f"/data/pj20/exp_data/ccscm_ccsproc/attention_weights_{task}.pkl"
attn = np.ndarray(shape=(len(clusters), 1))

for i in range(len(clusters)):
    idx = str(i)
    attn[i] = clusters[idx][f'attention_{task}']

with open(attn_file, "wb") as f:
    pickle.dump(attn, f)
